# 情報計算科学の基礎 
レポート2

$d^2u/dx^2 = 0$

$u=0@x=0, u=1@x=1$



In [2]:
import numpy as np
import math

def gausselimination_withpivot(n,A,b):
  x = np.zeros(n)
  Aorg = np.copy(A)
  borg = np.copy(b)

  #前進消去
  for j in range(n):
    #ピポット選択
    if A[j][j] == 0:
      list = [A[i][j] for i in range(j+1,n)]
      abs_list = [abs(i) for i in list]
      m = abs_list.index(max(abs_list))
      p = m + j + 1
      #交換
      b[j],b[p] = b[p],b[j]
      for c in range(n):
        A[j][c],A[p][c] = A[p][c],A[j][c]

    for i in range(j+1,n,1):
      coe=-A[i][j]/A[j][j]
      for k in range(j,n,1):
        A[i][k]+=coe*A[j][k]
      b[i]+=coe*b[j]
    
  #後退代入・結果出力
  for j in range(n-1,-1,-1):
    btmp=b[j]
    for i in range(n-1,j,-1):
      btmp -= A[j][i]*x[i]
    x[j]=btmp/A[j][j]
  return x



def create_K_elem(node_from,node_to):
  ds = abs(node_to - node_from)
  return np.array(([1/ds,-1/ds],[-1/ds,1/ds]))


def fill_K_glob(coord:list,N_SIZE_in:int):
    K_glob_in = np.zeros((N_SIZE_in,N_SIZE_in))
    K_elems = list()
    for i in range(N_SIZE_in - 1):
        K_elems.append(create_K_elem(coord[i],coord[i+1]))
        K_glob_in[i][i]     += K_elems[i][0][0]
        K_glob_in[i][i+1]   += K_elems[i][0][1]
        K_glob_in[i+1][i]   += K_elems[i][1][0]
        K_glob_in[i+1][i+1] += K_elems[i][1][1]
    return K_glob_in

def solve_with_cricon(N_SIZE_in:int,K_glob_in,b_in,U_init:list):
    K_del = np.delete(K_glob_in,[0,-1],0)
    # print(K_del)
    b_del = b_in[1:-1]
    # print(b_del)
    u_known = np.array(U_init)
    # print(u_known)
    K_known = np.delete(K_del,slice(1,-1,1),1)
    K_unknown = np.delete(K_del,[0,-1],1)
    # print('K_known')
    # print(K_known)
    # print('K_unknown')
    # print(K_unknown)

    b_prime = b_del - (K_known @ u_known)
    # print(b_prime)

    u = gausselimination_withpivot(N_SIZE_in-2,K_unknown,b_prime)
    print(u)
    # u.insert(0,U_0)
    # u.append(U_LAST)
    return np.insert(np.insert(u,len(u),U_init[1]), 0, U_init[0])


1. u(x) の理論解を求めよ。

$ u = x $ 
が理論会となる

In [3]:
U_0,U_LAST = 0,1

2. Fig. 1 のメッシュ(一次元の線形要素) を使って u(x) を求め、解析解と比較
せよ。


In [4]:
COORD = [0.00,0.25,0.50,1.0]
N_SIZE = len(COORD)

K_glob = fill_K_glob(COORD,N_SIZE)
print(K_glob)

answer = solve_with_cricon(N_SIZE,K_glob,np.zeros(N_SIZE),[U_0,U_LAST])

print(answer)



# u = gausselimination_withpivot(N_SIZE,K_glob,np.zeros(N_SIZE))




[[ 4. -4.  0.  0.]
 [-4.  8. -4.  0.]
 [ 0. -4.  6. -2.]
 [ 0.  0. -2.  2.]]
[0.25 0.5 ]
[0.   0.25 0.5  1.  ]


これは、理論解と一致する。
(結果の配列=COORD となっているので、　x=yとなっている)

3. Fig. 2 のメッシュ(一次元の線形要素) を使って u(x) を求め、解析解と比較
せよ。


In [5]:
COORD = [x/10 for x in range(0,11,1) ]
N_SIZE = len(COORD)

print(COORD)

K_glob = fill_K_glob(COORD,N_SIZE)
# print(K_glob)

answer = solve_with_cricon(N_SIZE,K_glob,np.zeros(N_SIZE),[U_0,U_LAST])

print(answer)

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


これも、理論解と一致する。

4. 加点対象（必須ではないがあれば加点します）：Fig. 3 のメッシュ(一次元の
線形要素) を使って u(x) を求め、解析解と比較せよ。Fig. 1, 2 のメッシュ
のように節点番号・要素番号が昇順に並んでいた場合と比べ、どのように
データ構造・プログラムを変更する必要があったか説明せよ。


まず、coordの順番を変えず、間隔のみ変更したコードを実行する。

In [6]:
COORD = [0, 0.125, 0.3, 0.45, 0.55, 0.6, 0.75, 0.9, 1.0]
N_SIZE = len(COORD)

print(COORD)

K_glob = fill_K_glob(COORD,N_SIZE)
# print(K_glob)

answer = solve_with_cricon(N_SIZE,K_glob,np.zeros(N_SIZE),[U_0,U_LAST])

print(answer)

[0, 0.125, 0.3, 0.45, 0.55, 0.6, 0.75, 0.9, 1.0]
[0.125 0.3   0.45  0.55  0.6   0.75  0.9  ]
[0.    0.125 0.3   0.45  0.55  0.6   0.75  0.9   1.   ]


次に、coordの順番を変えたものを実行する。

In [ ]:
COORD_ORIG = [0, 0.9, 0.3, 0.125, 0.55, 0.45, 0.75, 0.6, 1.0]
MAPING = [
        [6,1],
        [0,3],
        [2,5],
        [5,4],
        [3,2],
        [7,6],
        [4,7],
        [1,8]
    ]

N_SIZE = len(COORD)

print(COORD)

K_glob = fill_K_glob(COORD,N_SIZE)
# print(K_glob)

answer = solve_with_cricon(N_SIZE,K_glob,np.zeros(N_SIZE),[U_0,U_LAST])

print(answer)